# Creek analysis of sea level rise

Whitebox Tools Manual: <a href='https://jblindsay.github.io/wbt_book/intro.html'>Link</a>

Developed with the Anaconda Python distribution: <a href='https://anaconda.org'> www.anaconda.org</a>

To install rasterio and whitebox, type in the terminal   
`conda install -c conda-forge whitebox`  
`conda install -c conda-forge rasterio`  

For movie making, install the ffmpeg moviemaker: <a href='https://ffmpeg.org'> ffmpeg.org </a>

In [ ]:
##################################################################
# Import essential packages
##################################################################

import numpy as np
import matplotlib.pyplot as plt

import imageio
import rasterio
from rasterio.transform import from_origin
import time

%matplotlib inline
on=1;off=0

# Importing the Whitebox tools
import whitebox
wbt = whitebox.WhiteboxTools()

##################################################################
# Set working path
##################################################################

root_dir = '/Volumes/T7/GC_Hin_Reduce/Root/'
data_dir = '/Users/gccheng/CreekAnalyze/HinReduce/Data/'

LengthX  = 200.0  # 100.0  Size of the domain in physical dimensions
LengthY  = 200.0  # 100.0  Size of the domain in physical dimensions

# set whitebox working directory
wbt.set_working_dir(data_dir)
wbt.verbose = False

##################################################################
# Function definition
##################################################################

def PlotImage(Image,title, cmap, clim=None):
    
    if isinstance(Image, str):
        Image = imageio.imread(Image)

    fig, ax = plt.subplots(1, 1, figsize=(15, 15))

    im=ax.imshow(Image, cmap=cmap)
    ax.set_title(title, fontsize=15)
    ax.set_axis_off()
    if clim!=None: im.set_clim(clim)

    f=fig.colorbar(im, ax=ax, fraction=0.046, pad=0.04, orientation='vertical')

def SaveAsGeoTiff(Map,Name):
    with  rasterio.open(Name, 'w', driver='GTiff',
                     height=Map.shape[0], width=Map.shape[1],
                     count=1, dtype=Map.dtype,
                     crs='+proj=latlong', transform=transform) as Im:
        Im.write(Map, 1)
        Im.close()

def SaveAsPng(input,output,cmap):
    D = imageio.imread(input)
    plt.imsave(output,(D/np.max(D)*256), cmap=cmap)

CoverThreshold=100

# def GetBasins(CurrentTime):
def GetBasins( ):

    global CoverStore, BasinNrMax
    # SaveAsGeoTiff(ss[:,:,CurrentTime],data_dir+'DEM.tif')
    SaveAsGeoTiff(ss[:,:],data_dir+'DEM.tif')

    ### Deriving the flow accumulation and pointer maps

    dem       = data_dir+"DEM.tif"
    out_dem   = data_dir+"filled_dem.tif"
    out_pntr  = data_dir+"filled_pntr.tif"
    out_accum = data_dir+"filled_accum.tif"
    out_type  = "ca"

    wbt.flow_accumulation_full_workflow(
        dem, 
        out_dem, 
        out_pntr, 
        out_accum, 
        out_type, 
        log=False, 
        clip=True, 
        esri_pntr=False);

    # Drainage Basins
    out_basins = data_dir + f'Basins.tif'

    wbt.basins(
        out_pntr, 
        out_basins, 
        esri_pntr=False)
    
    #Delineating the streams
    out_streams = data_dir + f'streams'+format(deltaHin)+'_'+format(repeat)+'.tif'
    threshold = 20

    wbt.extract_streams(
        out_accum, 
        out_streams, 
        threshold, 
        zero_background=True)

    # Rereading the basin and streams Tif images
    Image = imageio.imread(out_basins)
    Streams = imageio.imread(out_streams)
    
    # Obtaining basin sizes
    BasinNrs, BasinCover = np.unique(Image.astype(int), return_counts=True)

    # Sorting according to basin size
    Ind = np.argsort(-BasinCover)
    BasinNrs = BasinNrs[Ind]
    BasinCover = BasinCover[Ind]

    # Only Basins larger then CoverThreshold are included
    Ind=BasinCover>CoverThreshold
    BasinCover=BasinCover[Ind]
    BasinNrs = BasinNrs[Ind]

    Number_of_Basins=BasinCover.shape[0]
    
    # Plotting the Streams on the image
    Image[Streams==1]=1

    return Image, BasinCover    


##################################################################
# Main part
##################################################################

NumFrames = 49 # NumFrames from 0-49 is stable, NumFrames from 50-99 is degradation

for i in range(10,90,10):
    deltaHin = i
    for repeat in range(1,6):
        
        filename = 'HinReduce_'+format(deltaHin)+'_'+format(repeat)+'_LastStep.npz'
        print(filename)
        
        Data = np.load(root_dir+filename)
        # us = Data['u']
        # vs = Data['v']
        # hs = Data['h']
        ss = Data['s']
        # bs = Data['b']
        # ds = Data['d']
        # ss = ss[0::2,0::2,NumFrames]
        ss = ss[0::2,0::2]

        Xdim,Ydim = ss.shape
        
        # Setting up coordinates
        x = np.linspace(-LengthX/2, LengthX/2, Xdim)
        y = np.linspace(-LengthY/2, LengthY/2, Ydim)
        X, Y = np.meshgrid(x, y)
        res = (x[-1] - x[0]) / Xdim
        # Creating a transformation
        transform = from_origin(x[0] - res / 2, y[-1] + res / 2, res, res) 
        
        Image, BasinCover = GetBasins( )
        
        PlotImage(Image,'Basins', cmap='prism')
        
        # Plot DEM image
        fig, ax = plt.subplots(1,1, figsize=(30, 15))
        im1=ax.imshow(ss[:,:], animated=True, interpolation='bilinear')
        ax.set_axis_off()
        plt.tight_layout()
        fig.savefig(data_dir + 'DEM'+format(deltaHin)+'_'+format(repeat)+'.tif',dpi = 100, bbox_inches = 'tight', pad_inches = 0)

In [ ]:
if (off):

    fig, ax = plt.subplots(1,1, figsize=(15, 4))

#     NetSpeed=np.sqrt(us[:,:,NumFrames-1]**2+vs[:,:,NumFrames-1]**2)

    # im1=ax.imshow(ss[:,:,i-1], animated=True, interpolation='bilinear')
    im1=ax.imshow(ss[:,:], animated=True, interpolation='bilinear')

#     ax.set_title('Sediment elevation [m]', y=1.02, fontsize=20)
    ax.set_axis_off()
#     f=fig.colorbar(im1, ax=ax, fraction=0.046, pad=0.04, orientation='vertical')
    
#     text1=fig.suptitle("Time: %1.0f of %1.0f" % (EndTime, EndTime), x=0.48, y=0.00, fontsize=16);
    
    plt.tight_layout()
    
    fig.savefig(data_dir + 'DEM'+format(deltaS)+'.tif',dpi = 500, bbox_inches = 'tight', pad_inches = 0)
    # fig.savefig(root_dir+'DEM{}.tif'.format(i))
#     fig.clf()

In [ ]:
# i = NumFrames-1
# Image, BasinCover = GetBasins(i)

# PlotImage(Image,'Basins', cmap='prism')

In [ ]:
if (off):

    fig, ax = plt.subplots(1,1, figsize=(15, 4))

#     NetSpeed=np.sqrt(us[:,:,NumFrames-1]**2+vs[:,:,NumFrames-1]**2)

    im1=ax.imshow(ss[:,:,i-1], animated=True, interpolation='bilinear')
#     ax.set_title('Sediment elevation [m]', y=1.02, fontsize=20)
    ax.set_axis_off()
#     f=fig.colorbar(im1, ax=ax, fraction=0.046, pad=0.04, orientation='vertical')
    
#     text1=fig.suptitle("Time: %1.0f of %1.0f" % (EndTime, EndTime), x=0.48, y=0.00, fontsize=16);
    
    plt.tight_layout()
    
    fig.savefig(data_dir + 'DEM{}.tif'.format(i),dpi = 500, bbox_inches = 'tight', pad_inches = 0)
    # fig.savefig(root_dir+'DEM{}.tif'.format(i))
#     fig.clf()

### Computing Cover for all timesteps

In [ ]:
# from ipywidgets import FloatProgress
# from IPython.display import display

# # Setting up a progress bar for the simulation
# print("Progress :");
# PB = FloatProgress(min=0, max=NumFrames); display(PB) 

# start_time = time.time() # Starting a timer:

# CoverStore=np.zeros((Tdim,1000))

# for i in range(Tdim):
#     Image, BasinCover =  GetBasins(i)
    
#     CoverStore[i,range(BasinCover.shape[0])]=BasinCover
    
#     PB.value += 1 # signal to increment the progress bar
       
# print(" Simulation took      : %1.1f (s)" % (time.time() - start_time))

### Extracting the filled portion of the CoverStore database

In [ ]:
# BasinNr=CoverStore.sum(axis=0)
# BasinNr=BasinNr[BasinNr>0]
# MaxBasins = BasinNr.shape[0]

# import numpy.matlib
# x=np.matlib.repmat(np.linspace(1,EndTime,Tdim), MaxBasins,1).T

# CoverStore=CoverStore[:,range(MaxBasins)]

### Plotting the area analysis results

In [ ]:
# fig, ax = plt.subplots(1, 2, figsize=(21, 9))

# im=ax[0].imshow(Image, animated=True, cmap='prism')
# ax[0].set_title('Basins', fontsize=20)
# ax[0].set_axis_off()

# f=fig.colorbar(im, ax=ax[0], fraction=0.046, pad=0.04, orientation='vertical')

# pl=ax[1].plot(x.ravel(), CoverStore.ravel(), 'bo', markersize=1, animated=True)
# ax[1].set_title('Basin sizes', fontsize=20)
# ax[1].set_xlabel('Recorded time step')
# ax[1].set_ylabel('Basin size');
# ln=ax[1].plot((0,0),(0,80000),color='red');

### Making an animation

In [ ]:
# from matplotlib import animation, rc

# def updatefig(i): # To update the image at each iteration

#     Image, BasinCover =  GetBasins(i)

#     im.set_array( Image )
#     ln[0].set_xdata((i/(NumFrames-1)*EndTime,i/(NumFrames-1)*EndTime))

#     return im,ln

# ani = animation.FuncAnimation(fig, updatefig, interval=100, frames=NumFrames, repeat=False)

# from IPython.display import HTML
# HTML(ani.to_html5_video())

### Saving the animation

In [ ]:
# # write to an mp4 movie
# Writer = animation.writers['ffmpeg']
# writer = Writer(fps=10, metadata=dict(artist='Guang-Cheng'), bitrate=5000)
# ani.save(root_dir+'Basins.mp4', writer=writer, dpi=120)

The End, Johan van de Koppel 2019